# Classes and types in python

Python is actually a complicated language when it comes to classes.  It has true multiple inheritance, unlike Java where  
a class may only extend one class, but implement many interfaces. Not only does it support true multiple inheritance,  
but it also has a different form of subtyping than traditional OOP languages. 

> Users of other languages sometimes get confused about how to implement OOP in python because it doesn't have   
> interfaces, traits or typeclasses.  Also, the infamous "duck typing" has a formal type equivalent called structural  
> subtyping which is different than nominative typing (ie, inheritance trees) like in java

So, to help understand all this, this lesson will cover the basics of python classes and its type system

Basics 

- Class vars vs Instance vars
- What is `object`
- Unions
- What is a `@dataclass`
    - Prefer classes as `just immutable data`
- Avoid inheritance trees and prefer composition, mixins, and protocols instead

Advanced

- How does multiple inheritance work?
    - method resolution order
- What is the equivalent of an `interface` in python then?
    - ABC and abstractmethod
- A brief mention of duck typing and Protocols


## Class vars

Class variables are similar to Java static fields in that they belong to the class, and not instances.  Be careful 
however, if you annotate them.  If you only annotate, and do not provide values, you actually haven't created class vars
and have actually only added annotations

In [ ]:
from pprint import pprint

# Example of an empty class.  You may think it has class variables `region` and `bucket`, but you dont
class S3Path:
    region: str
    bucket: str

pprint(dir(S3Path))
S3Path.__dict__

### Remember scoping!

The reason I taught about namespaces and scoping previously, was because namespaces applies to classes as well as
functions.

Run the code below, uncomment line 5 and run again.  Then try to understand why it failed

In [ ]:
s3_1 = S3Path()

s3_1.region = "tca"
# Uncomment this, to see an error
#print(S3Path.region)

# So how come the above failed?  Didn't s3_1.region set the class's region?
# To understand, look at all the fields of s3_1 by uncommenting the line below
#dir(s3_1)

# Now uncomment the below, and you will see it has a region field.
#s3_1.__dict__
#getattr(s3_1, "region")


### Classes in python are dynamic

To get a better handle on namespacing with classes, let's dynamically add fields

> Please don't do this in real code.  I am showing you this so that you can "monkey patch" code.  About the only place  
this should ever be done is when you need to mock out something for testing

In the code above, when we did `s3_1.region = "tca"`, what we really did, was add the key "region" to `s3_1` hidden  
`__dict__` field.  Essentially, the `s3_1.region` is in the `local` namespace of the instance.  It is not "shared" with  
the field `region` inside the namespace of the class `S3Path`.  Python lets you dynamically build up the fields of the  
class and instance.  This is a bad idea, but python lets you do it (it's also a huge reason why python is so slow)

In [ ]:
# A class that basically has and does nothing
class Foo:
    ...

f = Foo()
type(f)
f.blah = "hi there"
print(f.blah)
for i in dir(Foo):
    print(i)

def outside_method(self, count: int):
    self.count = count
    return self.count * 2

Foo.outside_method = outside_method
dir(Foo)
f.outside_method(10)

## The object type

In python, you can think of `object` like `Object` in Java.  It's the ancestor of all class types.  A long time ago in
2.7 and earlier, there used to be `Old` classes vs `New` classes.  To make a `New` type class, you had to subclass all
your class types like this:

```python
class ExampleNewType(object):
    ...
```

This is no longer needed since the 3.x days.

Note that `object` instances are immutable in a sense.  You can not dynamically add fields to them

In [ ]:
f = object()
type(f)

# Notice that there is not a __dict__ field, so you can't add any new fields to it
dir(f)
#f.age = 10

## Create class variables

In order to create class vars with annotations, you need to give them a default value, or create a `def __new__` 
method to initialize them.

> I will not cover `__new__` vs `__init__` as that is a different topic.  `__new__` is rarely used unless you are  
creating metaclasses (which is rare), you need to set class vars, or you somehow need to control that an object will be  
created at all.  `__init__` is called when the object has been instantiated in memory, but its instance fields need to  
to be set.  `__new__` controls creation of the class itself, and in turn calls `__init__`

In [ ]:
# In order to create class variables (eg, similar to a Java static), you need to assign a value
class S3Path:
    region: str
    bucket: str = "inin-tca"

    def __init__(self, region: str):
        self.region = region
        #S3Path.region = self.region.upper()

s3_1 = S3Path("tca")
# What do you think this will print if uncommented?
#print(S3Path.region)

# Uncomment the line S3Path.region in the __init__ above then uncomment this and run
#print(s3_1.region)


# s3_2 = S3Path("tca")
# s3_2.region = "dca"
# print(f"Now, s3_1.region = {s3_1.region} and S3Path.region = {S3Path.region}"")
# s3_1.region = "test"

## Instance fields

Much more common than class vars, are instance vars (or fields).  This is the bread and butter of most class types.
Let's redesign the `S3Path` with instance fields instead

In [ ]:
class S3Path:
    def __init__(self, region: str, bucket: str):
        self.region = region
        self.bucket = bucket

s3_1 = S3Path("test", "inin-tca")
print(s3_1)


### Dataclass class

Python has recognized that very often, what you really want is _just data_. Python has a builtin decorator called a  
dataclass that can perform some useful optimizations for you.  Unfortunately, it also makes it a bit confusing compared 
to regular classes.

In [ ]:
from dataclasses import dataclass


@dataclass
class S3Path:
    # These are no longer annotations on the class S3Path, but are actually the types of the instance
    # the decorator modifies the S3Path class, so that these are on the instances, 
    region: str
    bucket: str

#S3Path.__dict__

s3_1 = S3Path("test", "inin-tca")
print(s3_1)
s3_2 = S3Path("test", "inin-tca")
print(s3_1 == s3_2)
id(s3_1) == id(s3_2)


In [ ]:
from dataclasses import dataclass, field


@dataclass(kw_only=True)
class S3PathV2:
    region: str
    bucket: str = field(init=False)

    def __post_init__(self):
        self.bucket = f"inin-{self.region}"


# s3_1 = S3PathV2("test", "inin-tca")
# print(s3_1)
# s3_2 = S3Path("test", "inin-tca")
# s3_1 == s3_2